# ThreadDB Usage Demo

Welcome to ThreadDB. Here is a brief demo of some of the basic usage of ThreadDB. If you’re running this code in NodeJS, it will automatically include an IndexedDB polyfill to store data in SQLite, otherwise (in a browser) it should leverage the embedded IndexedDB database.

In [1]:
import { Database } from "@textile/threaddb";
import { schema, Person } from "./schema";

## Local First

One of the design constraints we set for ourselves when building ThreadDB was to ensure that a developer or user could get up and going with ThreadDB without an internet connection, and without any prior orchestration with a remote. So to create a new Database is as easy as:

In [2]:
// Create an empty db, with a defined schema, and open it
const db = await new Database("demo", { name: "Person", schema })
    .open(1); // Versioned dbs

We make a few assumptions to make this easy. 1. That the developer knows the schema(s) that they want to use for their Collections ahead of time, and 2. That a change in schemas will result in a change in the database version (so we also have versioned databases, thanks to Dexie).

Now, we just start using it. Similarly to MongoDB you can get a given Collection, and you can insert data into it. In our case, it will do schema validation on the way in.


In [3]:
// Just start using it!
const Person = db.collection("Person")
await Person.insert({ name: "Carson", age: 37 });

[ '01EPFRZYRZCM3X21KBS89BWX3X' ]


Because it is often useful to create an Instance before inserting it into the database, you can do that via the `create` method on a Collection:

In [4]:
// We can create instances before committing them
const entity = Person.create({ name: "Other", age: 2 });
await entity.exists() // false
await Person.has(entity._id); // false

false


All instances have an _id property like in MongoDB, which in our case, is a ULID. Once we have an instance, it has many useful properties that allow us to save, check if it exists in the database, delete, etc.

In [5]:
await entity.save();
await entity.exists(); // true

true


As with any reasonable database, you can query it, grab all Instances, count things, etc. We provide some nice MongoDB style querying, but you also have access to the full suite of query tools provided by Dexie if you want them:

In [6]:
await Person.find({}).count(); // Should be at least 2

2


Similarly, things like transactions are supported by ThreadDB. We have readTransaction and writeTransaction support, which matches our Go implementation to some degree. These are useful for batch inserts and for providing proper isolation and automaticity guarantees:


In [7]:
const person = { name: "Someting", age: 4 }
await Person.writeTransaction(async function () {
  const [id] = await Person.insert(person);
});

By the way, `insert` and many other methods take a variadic list of inputs, which makes it easy to do bulk inserts even when not using a transaction explicitly.

And of course, more complex queries are possible thanks to the (subset of) MongoDB query language that ThreadDB supports:

In [8]:
const people = Person.find({
  $or: [{ age: { $gt: 2 } }, { name: { $eq: "Something" } }],
} as any); // Should still find all 3 people
await people.toArray();

[
  DocumentInstance {
    name: 'Carson',
    age: 37,
    _id: '01EPFRZYRZCM3X21KBS89BWX3X'
  },
  DocumentInstance {
    name: 'Someting',
    age: 4,
    _id: '01EPFS25679HZ7PQW0M5EGD8K1'
  }
]


## Remotes

So far, we’ve only be doing local operations. Now it’s time to interact with a remote daemon. If you aren’t familiar with Textile’s Thread daemons, we recommend you [read up on them here](https://github.com/textileio/go-threads). The protocols and design of ThreadDB can be explored in detail in the white-paper: [A protocol & event-sourced database for decentralized user-siloed data](https://docsend.com/view/gu3ywqi). For further technical details. the reference implementation of Threads is written in Go and the full implementation details can be found [on godocs](https://godoc.org/github.com/textileio/go-threads). But for now, let’s simply connect to the Hub’s remote Threads daemon, so we don’t have to worry about running our own.

For working against the Hub, you’ll need a developer key, see the links above for some examples on how to generate this. We recommend starting with an insecure key for developing locally, before upgrading to a production setup.


> Please remember to use your own keys, rather than the demos keys provided in the Notebook!

In [9]:
// Set key info (this is an insecure key)
const key = "" // Replace with yours
const remote = await db.remote.setKeyInfo({ key })

Once we’ve “set” our remote, it is pretty easy to start working against it. But there are two things we need to do before we can start pushing data. The first, is to authenticate our local “user” or database instance, with the remote. This is required, regardless of the remote Threads daemon we are connecting with, be it the Hub, or our own daemon running on our laptop. Authentication is always against a public key (defaulting to an ED25519 signing pair). ThreadDB makes this super easy if you are working with our default key objects:

In [ ]:
import { PrivateKey } from "@textile/crypto";

// New random identity
const privateKey = PrivateKey.fromRandom()
// Grab the token, save it, or just use it
const token = await remote.authorize(privateKey);
console.log(token)

The response from `remote.authorize` is a token string. This is actually automatically added to the remote’s metadata under the hood, but since this token doesn’t expire, a developer might wish to store this in the user’s localStorage, or otherwise cache this information. You could even store the token within the user’s database if you wanted, though be advised, you should probably encrypt it if you are going to persist it anywhere insecure.

Now we’re ready to start working against the remote. The first thing you’ll likely want to do, is initialize a new database on the remote. This is essentially allocating a new Thread, and pushing the local schema information to the remote database. In practice, you’ll likely want to do this the first time a user connects with the remote daemon, and only then. But ThreadDB makes this operation pretty much idempotent, so if you accidentally try to initialize twice, you shouldn’t end up with more than one database on the remote. We’ll be working further to make this easier, so that it will detect version changes and things like that to handle schema changes “on the fly”.

In [ ]:
const id = await remote.initialize(); // Create random thread
console.log(id)

If course, if you already have a Thread/DB in mind, you can provide that string to the initialize method. This is a good idea if you want to invite another peer to your Thread that was created by a different peer, or if you have a “static” Thread that all users are going to interact with.

Now you just push…

In [12]:
await remote.push("Person");

Assuming no conflicts or issues with connecting to the remote, you’re off to the races. We can actually use our existing Threads Client library to validate that our changes were indeed pushed:

In [13]:
import { Client } from "@textile/threads-client"
import { ThreadID } from "@textile/threads-id"

const client = await Client.withKeyInfo({ key })
// Grab context just for our demo, not really needed
const context = client.context.withToken(token)
const found = await client.find(ThreadID.fromString(id), "Person", {})
console.log(found)

[
  { name: 'Carson', age: 37, _id: '01EPFRZYRZCM3X21KBS89BWX3X' },
  { _id: '01EPFS0NBK2QFA6SY1ZEECYDHE', name: 'Other', age: 2 },
  { name: 'Someting', age: 4, _id: '01EPFS25679HZ7PQW0M5EGD8K1' }
]


If all went according to plan, you should have the same set of instances on the remote as you do locally. Now try pushing more updates, or creating updates directly on the remote and pulling them into your local state. The remote API also has tools to “stash”, and “rebase” local changes on top of remote changes, and all sorts of additional tooling. This is what you should start playing with. Break it, we’ll fix it, and we’ll all build something amazing together.